### MOT metrics

In [1]:
import numpy as np
import glob
import os
import pandas as pd
import logging
from collections import OrderedDict
from pathlib import Path
import motmetrics as mm

In [2]:
# List all default metrics
mh = mm.metrics.create()
print(mh.list_metrics_markdown())

Name|Description
:---|:---
num_frames|Total number of frames.
obj_frequencies|Total number of occurrences of individual objects over all frames.
pred_frequencies|Total number of occurrences of individual predictions over all frames.
num_matches|Total number matches.
num_switches|Total number of track switches.
num_transfer|Total number of track transfer.
num_ascend|Total number of track ascend.
num_migrate|Total number of track migrate.
num_false_positives|Total number of false positives (false-alarms).
num_misses|Total number of misses.
num_detections|Total number of detected objects including matches and switches.
num_objects|Total number of unique object appearances over all frames.
num_predictions|Total number of unique prediction appearances over all frames.
num_unique_objects|Total number of unique object ids encountered.
track_ratios|Ratio of assigned to total appearance count per unique object id.
mostly_tracked|Number of objects tracked for at least 80 percent of lifespan.
par

### Evaluate MOT metrics

Functions to compute motchallenge metrics

In [15]:
def compare_dataframes(gts, ts):
    accs = []
    names = []
    for k, tsacc in ts.items():
        if k in gts:
            logging.info('Comparing {}...'.format(k))
            accs.append(mm.utils.compare_to_groundtruth(gts[k], tsacc, 'iou', distth=0.5))
            names.append(k)
        else:
            logging.warning('No ground truth for {}, skipping.'.format(k))

    return accs, names


def compute_motmetrics(groundtruths, tests, score_threshold, fmt='mot15-2D'):

    gtfiles = glob.glob(os.path.join(groundtruths, '*/gt/gt.txt'))
    print('gt_files', gtfiles)
    tsfiles = [f for f in glob.glob(os.path.join(tests, '*.txt')) if not os.path.basename(f).startswith('eval')]

    print('Found {} groundtruths and {} test files.'.format(len(gtfiles), len(tsfiles)))
    print('Available LAP solvers {}'.format(mm.lap.available_solvers))
    print('Default LAP solver \'{}\''.format(mm.lap.default_solver))
    print('Loading files.')

    gt = OrderedDict([(Path(f).parts[-3], mm.io.loadtxt(f, fmt=fmt, min_confidence=1)) for f in gtfiles])
    ts = OrderedDict(
        [(os.path.splitext(Path(f).parts[-1])[0], mm.io.loadtxt(f, fmt=fmt, min_confidence=score_threshold))
         for f in tsfiles])
    #     ts = gt

    mh = mm.metrics.create()
    accs, names = compare_dataframes(gt, ts)

    logging.info('Running metrics')
    metrics = ['recall', 'precision', 'num_unique_objects', 'mostly_tracked',
               'partially_tracked', 'mostly_lost', 'num_false_positives', 'num_misses',
               'num_switches', 'num_fragmentations', 'mota', 'motp', 'idf1', 'num_objects']
    summary = mh.compute_many(accs, names=names, metrics=metrics, generate_overall=True)
    div_dict = {
        'num_objects': ['num_false_positives', 'num_misses', 'num_switches', 'num_fragmentations'],
        'num_unique_objects': ['mostly_tracked', 'partially_tracked', 'mostly_lost']}
    for divisor in div_dict:
        for divided in div_dict[divisor]:
            summary[divided] = (summary[divided] / summary[divisor])
    fmt = mh.formatters
    change_fmt_list = ['num_false_positives', 'num_misses', 'num_switches', 'num_fragmentations', 'mostly_tracked',
                       'partially_tracked', 'mostly_lost']
    for k in change_fmt_list:
        fmt[k] = fmt['mota']
    print(mm.io.render_summary(summary, formatters=fmt, namemap=mm.io.motchallenge_metric_names))


### Calculate metrics for DeepSort MOT20

GT - directory with ground truth sequences.
DeepSort_predictions - directory with DeepSort predictions (*.txt files)

In [16]:
compute_motmetrics(groundtruths="GT", tests="DeepSort_Predictions", score_threshold=0.5, fmt='mot15-2D')

gt_files ['GT\\MOT20-01\\gt\\gt.txt', 'GT\\MOT20-02\\gt\\gt.txt', 'GT\\MOT20-03\\gt\\gt.txt', 'GT\\MOT20-05\\gt\\gt.txt']
Found 4 groundtruths and 4 test files.
Available LAP solvers ['scipy']
Default LAP solver 'scipy'
Loading files.
          Rcll  Prcn   GT   MT    PT    ML   FP    FN  IDs   FM  MOTA  MOTP  IDF1 num_objects
MOT20-01 20.1% 89.5%   74 5.4% 29.7% 64.9% 2.4% 79.9% 0.1% 0.6% 17.7% 0.075 28.2%       19870
MOT20-02 22.0% 89.0%  270 3.7% 38.5% 57.8% 2.7% 78.0% 0.1% 0.6% 19.2% 0.074 28.0%      154742
MOT20-03  0.5% 77.7%  702 0.0%  0.4% 99.6% 0.2% 99.5% 0.0% 0.1%  0.4% 0.092  0.8%      313658
MOT20-05  0.4% 76.1% 1169 0.0%  0.2% 99.8% 0.1% 99.6% 0.0% 0.0%  0.3% 0.102  0.7%      646344
OVERALL   3.8% 87.6% 2215 0.6%  5.9% 93.5% 0.5% 96.2% 0.0% 0.1%  3.2% 0.077  5.8%     1134614


### Calculte metrics for SORT MOT20

sort/output - directory with SORT predictions (*.txt files)

In [17]:
compute_motmetrics(groundtruths="GT", tests="sort/output", score_threshold=0.5, fmt='mot15-2D')

gt_files ['GT\\MOT20-01\\gt\\gt.txt', 'GT\\MOT20-02\\gt\\gt.txt', 'GT\\MOT20-03\\gt\\gt.txt', 'GT\\MOT20-05\\gt\\gt.txt']
Found 4 groundtruths and 4 test files.
Available LAP solvers ['scipy']
Default LAP solver 'scipy'
Loading files.
          Rcll  Prcn   GT    MT    PT    ML   FP    FN  IDs   FM  MOTA  MOTP  IDF1 num_objects
MOT20-01 54.1% 90.6%   74 23.0% 52.7% 24.3% 5.6% 45.9% 0.9% 1.2% 47.7% 0.089 48.1%       19870
MOT20-02 50.2% 91.5%  270 21.5% 61.1% 17.4% 4.6% 49.8% 0.7% 1.0% 44.8% 0.081 37.5%      154742
MOT20-03 45.8% 94.1%  702 10.3% 59.7% 30.1% 2.9% 54.2% 1.3% 2.1% 41.6% 0.137 35.1%      313658
MOT20-05 47.2% 90.7% 1169 12.2% 60.7% 27.1% 4.8% 52.8% 1.2% 1.9% 41.2% 0.127 30.7%      646344
OVERALL  47.3% 91.7% 2215 13.1% 60.1% 26.8% 4.3% 52.7% 1.1% 1.8% 41.9% 0.122 33.1%     1134614


With SORT we have much better results

### Calculating metrics for IOU-tracker MOT20

iou-tracker/output contains predictions in *.txt format

In [18]:
compute_motmetrics(groundtruths="GT", tests="iou-tracker/output", score_threshold=0.5, fmt='mot15-2D')

gt_files ['GT\\MOT20-01\\gt\\gt.txt', 'GT\\MOT20-02\\gt\\gt.txt', 'GT\\MOT20-03\\gt\\gt.txt', 'GT\\MOT20-05\\gt\\gt.txt']
Found 4 groundtruths and 4 test files.
Available LAP solvers ['scipy']
Default LAP solver 'scipy'
Loading files.
          Rcll  Prcn   GT   MT    PT    ML   FP    FN  IDs   FM  MOTA  MOTP  IDF1 num_objects
MOT20-01 17.8% 90.2%   74 5.4% 23.0% 71.6% 1.9% 82.2% 0.4% 0.4% 15.5% 0.059 19.7%       19870
MOT20-02 20.3% 89.3%  270 3.0% 37.0% 60.0% 2.4% 79.7% 0.4% 0.4% 17.4% 0.059 20.4%      154742
MOT20-03  0.3% 74.8%  702 0.0%  0.3% 99.7% 0.1% 99.7% 0.0% 0.0%  0.2% 0.077  0.2%      313658
MOT20-05  0.3% 73.5% 1169 0.0%  0.2% 99.8% 0.1% 99.7% 0.0% 0.0%  0.2% 0.075  0.4%      646344
OVERALL   3.3% 88.0% 2215 0.5%  5.5% 94.0% 0.5% 96.7% 0.1% 0.1%  2.8% 0.060  3.9%     1134614


SORT algorithm appeared to be the best among others, then goes DeepSort and then IOU-tracker.